In [1]:
# Импорт библиотек, необходимых для NLP
import nltk
nltk.download("punkt")
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Импорт библиотек, необходимых для обработки Tensorflow
import tensorflow as tf   
import numpy as np
import tflearn           
import random
import json

[nltk_data] Downloading package punkt to /home/andor/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/andor/.local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/andor/.local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/home/andor/.local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/andor/.local/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/home/andor/.local/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/home/andor/.

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!pip install nltk
!pip install tensorflow
!pip install numpy
!pip install tflearn
!pip install random
!pip install json

In [21]:
# импортируем наш файл intents.json, используемый для обучения модели.
with open("/home/andor/Рабочий стол/5.5/intents.json") as json_data: 
    intents = json.load(json_data)      # Загрузка данных из файла intents.json в var

In [22]:
# Пустые списки для добавления данных после обработки NLP
words=[]
documents = []
classes = []


# Этот список будет использоваться для игнорирования всех нежелательных знаков препинания.
ignore = ["?"]

# Запуск цикла по каждому intents в intents["patterns"]
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        
        # токенизация каждого слова в предложении с помощью токенизатора слов и сохранение в w
        w = nltk.word_tokenize(pattern) 
        #print(w)
        
        # Добавление токенизированных слов в пустой список слов, который мы создали
        words.extend(w) 
        #print(words)
        
        # Добавление слов в документы с тегом, указанным в файле намерений intents
        documents.append((w, intent["tag"]))
        #print(documents)
        
        # Добавление только тега в список наших классов
        if intent["tag"] not in classes:      
            classes.append(intent["tag"])  #Если тег отсутствует в классах [], он будет добавлен к нему..
            #print(classes)
            

In [23]:
#Выполнение стемминга с помощью stemmer.stem()
#ЗАпуск повторений в  words[] и игнорирование знаков препинания, присутствующих в ignore[]

words = [stemmer.stem(w.lower()) for w in words if w not in ignore]  
words = sorted(list(set(words)))  #Removing Duplicates in words[]

#Удаление повторяющихся классов
classes = sorted(list(set(classes)))

#Длина печати сформированных нами списков
print(len(documents),"Documents \n")
print(len(classes),"Classes \n")
print(len(words), "Stemmed Words ")

12 Documents 

4 Classes 

32 Stemmed Words 


In [24]:
#Создание обучающих данных, которые в дальнейшем будут использоваться для обучения
training = []
output = []

#Creating empty array for output
output_empty = [0] * len(classes)

#Создание пустого массива для вывода
for doc in documents:
    bag = [] #Инициализация пустого списка слов

    pattern_words = doc[0] #Хранение списка токенизированных слов для документов[] tp pattern_words
    #print(pattern_words)
    
    #Снова вывод каждого слова из pattern_words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]  
    #print(pattern_words)
    
    #Создание массива списка слов
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #Это даст результат 1 для текущего тега и 0 для всех остальных тегов.
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] =1 
    training.append([bag, output_row])

In [25]:
random.shuffle(training) #Перетасовка данных или функций
training = np.array(training) #Преобразование данных обучения в массив numpy array

#Создание списков тренировок
train_x = list(training[:,0])
train_y = list(training[:,1])

/tmp/ipykernel_3261/3014420881.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training) #Преобразование данных обучения в массив numpy array


In [26]:
tf.compat.v1.reset_default_graph() #Сброс данных базового графика

#Создание собственной нейронной сети
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation="softmax")
net = tflearn.regression(net)

#Определение модели и настройкаp tensorboard
model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

#Теперь у нас есть настроенная модель, теперь нам нужно обучить эту модель, вписав в нее данные с помощью model.fit()
#n_epoch это сколько раз модель будет использовать ваши данные во время обучения
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True) 
model.save("model.tflearn") #Saving the model

Training Step: 1999  | total loss: 0.35243 | time: 0.002s
| Adam | epoch: 1000 | loss: 0.35243 - acc: 0.9484 -- iter: 08/12
Training Step: 2000  | total loss: 0.31771 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.31771 - acc: 0.9536 -- iter: 12/12
--
INFO:tensorflow:/home/andor/Рабочий стол/5.5/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [27]:
#Импорт модуля pickle
import pickle

#Сброс обучающих данных с помощью dump() и запись их в training_data в двоичном режиме
pickle.dump({"words":words, "classes":classes, "train_x":train_x, "train_y":train_y}, open("training_data", "wb"))

In [28]:
#Восстановление всей структуры данных
data = pickle.load(open("training_data","rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [29]:
with open("/home/andor/Рабочий стол/5.5/intents.json") as json_data:
    intents = json.load(json_data)  #Загрузка наших json_data

In [30]:
# Загрузка сохраненной модели
model.load("./model.tflearn") #Загрузка модели обучения, которую мы сохранили

INFO:tensorflow:Restoring parameters from /home/andor/Рабочий стол/5.5/model.tflearn


In [31]:
#Очистка пользовательского ввода
def clean_up_sentence(sentence):
    
    # Токенизация шаблона
    sentence_words = nltk.word_tokenize(sentence) #Again tokenizing the sentence
    
    #Получение каждого слова из ввода пользователя
    sentence_words= [stemmer.stem(word.lower()) for word in sentence_words]

    return sentence_words

#Возвращаемый массив массива слов: 0 или 1 или каждое слово в списке, которое существует, как мы объявили в приведенных выше строках.
def bow(sentence, words, show_details=False):
    
    #Токенизация пользовательского ввода
    sentence_words = clean_up_sentence(sentence)
    
    #Генерация набора слов из предложения, которое ввел пользователь
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("Found in bag: %s"% w)
    return(np.array(bag))

In [32]:
#Добавление некоторого контекста в разговор для лучших результатов..

context = {} #Создайте словарь для хранения контекста пользователя

ERROR_THRESHOLD = 0.25
def classify(sentence):
    
    #Генерация вероятностей из модели
    results = model.predict([bow(sentence, words)])[0]
    
    #Отфильтровать прогнозы ниже порогового значения
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    
    #Сортировка по силе вероятности
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    
    # вернуть кортеж намерения и вероятности
    return return_list

def query(sentence, userID='123', show_details=False):
    results = classify(sentence)
    
    #Если у нас есть классификация, найдите соответствующий тег намерения
    if results:
        
        #Цикл, пока есть совпадения для обработки
        while results:
            for i in intents['intents']:
                
                #Найти тег, соответствующий первому результату
                if i['tag'] == results[0][0]:
                    
                    #Установить контекст для этого намерения, если необходимо
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # проверить, является ли это намерение контекстуальным и применимо к разговору этого пользователя
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        
                        #Случайный ответ от намерения
                        return print(random.choice(i['responses']))

            results.pop(0)

In [33]:
query("Здравствуйте")

Авилон приветствует вас !


In [34]:
query("Вы не против, если мы будем снимать ?")

Вам нужно разрешение директора автосалона


In [35]:
query ("Автосалон это общественное место, а значит мы можем снимать")

Я же вам сказал, снимать нельзя


In [36]:
query ("Ладно, мы уходим")

Возращайтесь без камер.
